<a href="https://colab.research.google.com/github/mostafa-ja/Data-Privacy/blob/main/Preserving_Data_Privacy_in_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset 
torch.backends.cudnn.benchmark=True

In [14]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 6
num_rounds = 150
epochs = 5
batch_size = 32

**3. Loading and dividing CIFAR 10 into clients**

CIFAR10 dataset is used in this tutorial. It consists of 60,000 color images of 32x32 pixels in 10 classes. There are 50,000 training images and 10,000 test images. In the training batch, there are 5,000 images from each class, which makes 50,000 in total.

In this tutorial, images are equally divided into clients, thus representing the balanced (IID) case.



```
 generator = torch.Generator().manual_seed(42)
 random_split(range(10), [3, 7], generator=generator)

 train_data.data.shape[0]
```



In [15]:

#############################################################
##### Creating desired data distribution among clients  #####
#############################################################

# Image augmentation 
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Loading CIFAR10 using torchvision.datasets
train_data = datasets.CIFAR10('./train_data',train=True,download=True,
                              transform=transform_train)

# Dividing the training data into num_clients, with each client having equal number of images
train_data_split = torch.utils.data.random_split(train_data,[int(train_data.data.shape[0]/num_clients) for _ in range(num_clients) ])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x,batch_size=batch_size,shuffle=True) for x in train_data_split ]

# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading the test iamges and thus converting them into a test_loader
test_data = datasets.CIFAR10('./test_data',train=False,download=True,transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


**4. Building the Neural Network (Model Architecture)**

VGG19 (16 convolution layers, 3 Fully Connected layers, 5 MaxPool layers, and 1 SoftMax layer) are used in this tutorial. There are other variants of VGG like VGG11, VGG13, and VGG16.

In [16]:

#################################
##### Neural Network model #####
#################################

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
  def __init__(self,vgg_name):
    super(VGG,self).__init__()
    self.features = self.make_layers(cfg[vgg_name])
    self.classifier = nn.Sequential(
        nn.Linear(512,512),
        nn.ReLU(True),
        nn.Linear(512,512),
        nn.ReLU(True),
        nn.Linear(512,10)
    )

  def forward(self,x):
    out = self.feature(x)
    out = out.view(out.size(0),-1)
    out = out.classifier(out)
    output = F.log_softmax(out,dim=1)
    return output

  def make_layers(self,cfg):
    layers = []
    in_channels = 3
    for x in cfg:
      if x == 'M':
        layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
      else:
        layers += [nn.Conv2d(in_channels,x,kernel_size=3,padding=1),
                   nn.BatchNorm2d(x),
                   nn.ReLU(inplace=True)]
        in_channels = x
    layers += [nn.AvgPool2d(kernel_size=1,stride=1)]
    return nn.Sequential(*layers)


**5. Helper functions for Federated training**

The client_update function train the client model on private client data. This is the local training round that takes place at num_selected clients, i.e. 6 in our case.

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
def client_update(client_model,optimizer,train_loader,epoch=5):
    """
    This function updates/trains client model on client data
    """
    client_model.to(device)
    client_model.train()
    for e in range(epoch):
      for batch_idx, (data,target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = client_model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
    return loss.item()

The server_aggregate function aggregates the model weights received from every client and updates the global model with the updated weights. In this tutorial, the mean of the weights is taken and aggregated into the global weights.

In [ ]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###

    